### Neo4j Environment Setup

In [1]:
# Import libraries for environment variable management
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
# Extract Neo4j connection credentials from environment variables
# These credentials are used to establish connection to the Neo4j database
URL = os.environ.get("NEO4J_URI")  
USERNAME = os.environ.get("NEO4J_USERNAME")  
PASSWORD = os.environ.get("NEO4J_PASSWORD") 

In [3]:
# Initialize Neo4jGraph instance to connect to the graph database
# Neo4jGraph provides an interface to interact with Neo4j using LangChain
from langchain_neo4j import Neo4jGraph

graph = Neo4jGraph(
    url=URL,  
    username=USERNAME,  
    password=PASSWORD  
)
graph

### Loading Dataset into a Graph

In [ ]:
# Cypher query to load articles from CSV and create graph structure
# This query creates Article nodes, Researcher nodes, and their relationships
load_articles = """
LOAD CSV WITH HEADERS
FROM 'https://raw.githubusercontent.com/dcarpintero/generative-ai-101/main/dataset/synthetic_articles.csv' 
AS row 
FIELDTERMINATOR ';'

-- Create or merge Article nodes with title as unique identifier

MERGE (a:Article {title:row.Title})
SET a.abstract = row.Abstract,
    a.publication_date = date(row.Publication_Date)
    
-- Create Researcher nodes for each author and establish PUBLISHED relationship

FOREACH (researcher in split(row.Authors, ',') | 
    MERGE (p:Researcher {name:trim(researcher)})
    MERGE (p)-[:PUBLISHED]->(a))
    
-- Create Topic nodes and link articles to topics via IN_TOPIC relationship

FOREACH (topic in [row.Topic] | 
    MERGE (t:Topic {name:trim(topic)})
    MERGE (a)-[:IN_TOPIC]->(t))
"""

# Execute the Cypher query to populate the graph database
result = graph.query(load_articles)
result

[]

In [4]:
# Refresh the graph schema to reflect any changes from the loaded data
# This updates the internal representation of nodes, relationships, and their properties
graph.refresh_schema()

# Print the current schema to understand the structure of the graph database
print(graph.get_schema)

Node properties:
Article {title: STRING, abstract: STRING, publication_date: DATE, embedding: LIST}
Researcher {name: STRING}
Topic {name: STRING}
Relationship properties:

The relationships:
(:Article)-[:IN_TOPIC]->(:Topic)
(:Researcher)-[:PUBLISHED]->(:Article)


### Building a Vector Index

In [23]:
# Initialize LLM (Language Model) and Embedding Model for the RAG system
from langchain_groq import ChatGroq
from langchain_google_genai import GoogleGenerativeAIEmbeddings

llm = ChatGroq(model="openai/gpt-oss-20b") 
embedding_model = GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-001")

In [24]:
# Create a Neo4j Vector Index for hybrid search on Article nodes
# This combines dense (semantic) vector search with full-text search capabilities
from langchain_neo4j import Neo4jVector

vector_index = Neo4jVector.from_existing_graph(
    embedding=embedding_model,  # Use Gemini embeddings for vector representations
    index_name="articles",  # Name of the index in Neo4j
    node_label="Article",  # Create embeddings for Article nodes
    text_node_properties=['topic', 'title', 'abstract'],  # Properties to embed
    embedding_node_property="embedding",  # Property name to store embeddings
    search_type="hybrid",  # Use hybrid search (combines vector + full-text)
)

In [7]:
# Cypher query to verify that embeddings have been created for articles
# This checks that vector embeddings are properly stored in the database
cypher = """MATCH (a:Article)
WHERE a.embedding IS NOT NULL
RETURN a.title, size(a.embedding) AS vector_size
LIMIT 5;
"""

results = graph.query(cypher)
results

[{'a.title': 'Transformer Architecture Innovations', 'vector_size': 3072},
 {'a.title': 'Ethical Considerations in AI Development', 'vector_size': 3072},
 {'a.title': 'Optimizing Large Language Models for Edge Devices',
  'vector_size': 3072},
 {'a.title': 'The Impact of AI on Employment: A Comprehensive Study',
  'vector_size': 3072},
 {'a.title': 'Attention Mechanism Enhancements for Improved Language Understanding',
  'vector_size': 3072}]

In [8]:
# Create a retriever from the vector index to fetch relevant articles
retriever = vector_index.as_retriever(k=3)

In [9]:
# Test the retriever with a sample query
query = "which articles discuss how AI might affect our daily life? include the article titles and abstracts also give author name."
result = retriever.invoke(query)

In [10]:
# Shows the content, metadata, and relevance of the article
result[1]

Document(metadata={'publication_date': neo4j.time.Date(2023, 6, 28)}, page_content='\ntopic: \ntitle: The Role of AI in Combating Climate Change: Opportunities and Challenges\nabstract: Our research explores how AI can be leveraged to address climate change, discussing both its potential benefits and the associated ethical considerations.')

### Langchain-Agent

In [25]:
# Define a RAG (Retrieval-Augmented Generation) tool for the agent
# This tool retrieves relevant documents using semantic search before generating responses
from langchain_core.tools import tool

@tool
def rag_tool(query: str) -> dict:
    """
    Retrieve relevant information from the graph database using semantic search.
    Use this tool when the user asks factual or conceptual questions
    that might be answered from the stored documents.
    
    Args:
        query (str): The user's question or search query
    
    Returns:
        dict: Dictionary containing:
            - query: The original query
            - context: List of relevant document excerpts
            - metadata: Metadata information for each document
    """
    # Retrieve top 3 most similar documents for the query using vector similarity
    result = retriever.invoke(query, k=3)
    
    context = [doc.page_content for doc in result]
    metadata = [doc.metadata for doc in result]
    
    # Return structured response with query and retrieved context
    return {
        "query": query,
        "context": context,
        "metadata": metadata,
    }

In [26]:
# Define a Graph Query tool for executing Cypher queries on the knowledge graph
# This tool is used for counting, relationships, and structured data retrieval
from langchain_neo4j import GraphCypherQAChain

@tool
def graph_cypher_tool(query: str) -> str:
    """
    Execute graph queries using Cypher language for structured data retrieval.
    Use for questions about quantities, connections, relationships, or specific data retrieval
    that requires navigating the graph structure.
    
    Args:
        query (str): Natural language question about graph data
        
    Returns:
        str: The result of the Cypher query execution
    """
    # Create a GraphCypherQAChain that translates natural language to Cypher queries
    cypher_chain = GraphCypherQAChain.from_llm(
        llm=llm,  
        graph=graph,  
        allow_dangerous_requests=True,  
    )
    response = cypher_chain.invoke({"query": query})
    return response['result']

In [13]:
# Test the graph query using Cypher language directly
# This query counts the number of articles published by a specific researcher
cypher = """
MATCH (r:Researcher {name: "Emily Chen"})-[:PUBLISHED]->(a:Article)
RETURN COUNT(a) AS numberOfArticles
"""

r = graph.query(cypher)
r

[{'numberOfArticles': 7}]

In [27]:
# Test the Graph Cypher tool by asking a natural language question
# The tool will automatically convert this to Cypher and execute it
# Expected answer: 7 articles
graph_cypher_tool.invoke(
    "How many articles has published Emily Chen?"
)

'Emily Chen has published 7 articles.'

In [28]:
# Combine all tools available to the agent
tools = [rag_tool, graph_cypher_tool]

In [29]:
# Import agent creation utilities and memory management tools
from langchain.agents import create_agent
from langgraph.checkpoint.memory import MemorySaver

In [30]:
# Initialize memory saver for storing conversation state and agent history
memory = MemorySaver()

In [36]:
# Create a ReAct agent (Reasoning + Acting) with tool access
agent = create_agent(
    model=llm,  
    tools=tools,
    checkpointer=memory,
    system_prompt="You are a knowledge assistant with access to a graph database. Always base answers on retrieved context and cite sources with specific titles or identifiers when available.",
    # debug=True
)

In [37]:
# Set up configuration for agent execution with a specific thread ID
config = {"configurable": {"thread_id": "1"}}

In [38]:
def invoke_agent(query: str):
    result = agent.invoke(
        {"messages": query},
        config=config,
    )
    response = result['messages'][-1].content
    return response

In [39]:

result = agent.invoke(
    {"messages": "which articles discuss how AI might affect our daily life? include the article titles and abstracts also give author name."},
    config=config,
)

In [40]:
from IPython.display import Markdown, display
response = result['messages'][-1].content
display(Markdown(response))

I searched the database for articles that touch on how artificial‑intelligence (AI) is likely to influence everyday life.  The following papers were retrieved with their titles, abstracts, and (where available) the authors who wrote them.  Author information is not stored in the current graph, so it is not displayed here.

| Article title | Abstract | Source |
|---------------|----------|--------|
| **The Impact of AI on Employment: A Comprehensive Study** | “This study analyzes the potential effects of AI on various job sectors and suggests policy recommendations to mitigate negative impacts.” | “The Impact of AI on Employment: A Comprehensive Study” (2023‑11‑18) |
| **AI and Privacy: Balancing Innovation and Individual Rights** | “We examine the tension between AI advancement and privacy protection, proposing a balanced approach that fosters innovation while safeguarding personal data.” | “AI and Privacy: Balancing Innovation and Individual Rights” (2023‑10‑30) |
| **The Societal Implications of Advanced AI: A Multidisciplinary Analysis** | “Our study brings together experts from various fields to analyze the potential long‑term impacts of advanced AI on society, economy, and culture.” | “The Societal Implications of Advanced AI: A Multidisciplinary Analysis” (2023‑10‑11) |
| **Ethical Considerations in AI Development** | “We explore the ethical implications of rapid AI advancement and propose guidelines for responsible development.” | “Ethical Considerations in AI Development” (2023‑07‑23) |

These articles collectively address how AI can shape daily life—from changes in employment and privacy concerns to broader societal and ethical ramifications.  If you need author details or more specific information, let me know and I can try to fetch that data.

In [44]:
result = agent.invoke(
    {"messages": "How many articles has published Emily Chen? and list their titles."},
    config=config,
)

In [45]:
from IPython.display import Markdown, display
response = result['messages'][-1].content
display(Markdown(response))

Emily Chen has authored **7** articles.  
Below are the titles of those articles:

1. **Transformer Architecture Innovations**  
2. **Optimizing Large Language Models for Edge Devices**  
3. **Ensuring Robustness in AI Systems: A Safety‑First Approach**  
4. **Scaling Laws in Language Model Training: New Insights**  
5. **The Role of AI in Combating Climate Change: Opportunities and Challenges**  
6. **Multilingual Pretraining: Towards Universal Language Understanding**  
7. **Reinforcement Learning for Safe Exploration in Language Models**

### Reference
[Langchain Docs](https://docs.langchain.com/oss/python/integrations/graphs/neo4j_cypher#enhanced-schema-information) <br>
[LangChain Neo4j Integration](https://neo4j.com/labs/genai-ecosystem/langchain/) <br>
[HuggingFace - Enhancing RAG Reasoning with Knowledge Graphs](https://huggingface.co/learn/cookbook/en/rag_with_knowledge_graphs_neo4j#enhancing-rag-reasoning-with-knowledge-graphs)